# Noisy simulations on GPU

Noisy simulations running in "stochastic" mode can be offloaded to run on a GPU. As in the case of ideal simulations, to use the GPU based simulator, it is sufficient to set the fields "use_GPU" (to 'True') and "precision" (to either 1 or 2) while initializing the 'NoisyQProc'. In this notebook we show some use cases, where we obtain a speedup in running the simulations on a GPU. 

In [1]:
import numpy as np

from qat.core import Observable, Term
from qat.lang.AQASM import Program
from qat.lang.AQASM.qftarith import QFT
from qat.hardware import DefaultHardwareModel
from qat.quops.quantum_channels import ParametricAmplitudeDamping, ParametricPureDephasing
from qat.noisy.noisy_simulation import compute_fidelity
from qat.qpus import NoisyQProc

As an example, we consider the Quantum Fourier Transform (QFT) circuit and we suppose a simple noise model with idle qubits subject to parametric amplitude damping.

In [2]:
nqbits = 14
prog = Program()
reg = prog.qalloc(nqbits)
prog.apply(QFT(nqbits), reg)
circ = prog.to_circ()

hardware_model = DefaultHardwareModel(gate_times = {"H": 0.2, "C-PH": lambda angle:0.65},
                                      idle_noise = [ParametricAmplitudeDamping(T_1 = 75)])    

Here we choose to run the simulation with 1000 samples and initialize the NoisyQProc with different arguments to compare the output and their respective runtimes.

In [3]:
n_samples = 1000

noisy_qpu_gpu_single = NoisyQProc(hardware_model=hardware_model, sim_method="stochastic", 
                                  n_samples=n_samples, use_GPU=True, precision=1)
noisy_qpu_gpu_double = NoisyQProc(hardware_model=hardware_model, sim_method="stochastic", 
                                  n_samples=n_samples, use_GPU=True, precision=2)
noisy_qpu_cpu = NoisyQProc(hardware_model=hardware_model, sim_method="stochastic", n_samples=n_samples)

# I Fidelity of noisy QFT

In [4]:
%%time
fid_cpu, err_cpu = compute_fidelity(circ, noisy_qpu_cpu) # Simulation running on 1 node (cpu)
print(fid_cpu, err_cpu)

0.8904250448070108 0.011051199884490407
CPU times: user 27.1 s, sys: 2.71 s, total: 29.8 s
Wall time: 6.04 s


In [5]:
%%time
fid_gpu_single, err_gpu_single = compute_fidelity(circ, noisy_qpu_gpu_single) # Simulation running in single precision on a GPU
print(fid_gpu_single, err_gpu_single)

0.9032124235728615 0.01079993847196731
CPU times: user 2.48 s, sys: 334 ms, total: 2.81 s
Wall time: 1.59 s


In [6]:
%%time
fid_gpu_double, err_gpu_double = compute_fidelity(circ, noisy_qpu_gpu_double) # Simulation running in double precision on a GPU
print(fid_gpu_double, err_gpu_double)

0.8958888306364972 0.010975832297648416
CPU times: user 2.58 s, sys: 83.5 ms, total: 2.67 s
Wall time: 1.38 s


Here we compare the stochastic results with a deterministic evaluation and check the time it takes to get an exact value.

In [7]:
noisy_qpu_det = NoisyQProc(hardware_model=hardware_model, sim_method="deterministic-vectorized")

In [8]:
%%time
fid_cpu_det, _ = compute_fidelity(circ, noisy_qpu_det) # Deterministic simulation running on 1 node (cpu)
print(fid_cpu_det)

0.9076533148416498
CPU times: user 19min 33s, sys: 5.44 s, total: 19min 38s
Wall time: 37.4 s


# II Sampling a noisy QFT

In [9]:
job = circ.to_job(nbshots=100)

In [10]:
%%time
res_cpu = noisy_qpu_cpu.submit(job)

CPU times: user 42.7 s, sys: 251 ms, total: 42.9 s
Wall time: 6.96 s


In [11]:
%%time
res_gpu_single = noisy_qpu_gpu_single.submit(job)

CPU times: user 3.42 s, sys: 25 ms, total: 3.44 s
Wall time: 2.23 s


In [12]:
%%time
res_gpu_double = noisy_qpu_gpu_double.submit(job)

CPU times: user 3.61 s, sys: 144 ms, total: 3.76 s
Wall time: 2.51 s


# III Observable evaluation
 Here we generate a random observable with 40 terms and evaluate it

In [13]:
n_terms = 40
terms = ["X", "Y", "Z"]
pauli_terms = []
for _ in range(n_terms):
    term = ""
    for _ in range(np.random.choice([1, 2], 1)[0]):
        term += np.random.choice(terms, 1)[0]
    pauli_terms.append(Term(1.0, term, list(np.random.choice(nqbits, len(term), replace=False))))
    
obs = Observable(nqbits, pauli_terms=pauli_terms)

In [14]:
job_obs = circ.to_job("OBS", observable=obs)

In [15]:
%%time
res_cpu_obs = noisy_qpu_cpu.submit(job_obs)
print(res_cpu_obs.value, res_cpu_obs.error)

10.344816487988275 0.006628395717573745
CPU times: user 30.7 s, sys: 193 ms, total: 30.9 s
Wall time: 6.82 s


In [16]:
%%time
res_gpu_single_obs = noisy_qpu_gpu_single.submit(job_obs)
print(res_gpu_single_obs.value, res_gpu_single_obs.error)

10.367441184116432 0.006646644654630753
CPU times: user 1min 24s, sys: 127 ms, total: 1min 24s
Wall time: 3.12 s


In [17]:
%%time
res_gpu_double_obs = noisy_qpu_gpu_double.submit(job_obs)
print(res_gpu_double_obs.value, res_gpu_double_obs.error)

10.197279732916705 0.006735216986623743
CPU times: user 1min 37s, sys: 89.9 ms, total: 1min 37s
Wall time: 3.65 s
